# 12-01. RAG 기본 구조 이해하기

## 1. 사전작업(Pre-processing) - 1~4 단계

사전 작업 단계에서는 데이터 소스를 Vector DB (저장소) 에 문서를 로드-분할-임베딩-저장 하는 4단계를 진행합니다.

- 1단계 문서로드(Document Load): 문서 내용을 불러옵니다.
- 2단계 분할(Text Split): 문서를 특정 기준(Chunk) 으로 분할합니다.
- 3단계 임베딩(Embedding): 분할된(Chunk) 를 임베딩하여 저장합니다.
- 4단계 벡터DB 저장: 임베딩된 Chunk 를 DB에 저장합니다.

## 2. RAG 수행(RunTime) - 5~8 단계

- 5단계 검색기(Retriever): 쿼리(Query) 를 바탕으로 DB에서 검색하여 결과를 가져오기 위하여 리트리버를 정의합니다. 리트리버는 검색 알고리즘이며(Dense, Sparse) 리트리버로 나뉘게 됩니다. 
  - **Dense**: 유사도 기반 검색(FAISS, DPR)
  - **Sparse**: 키워드 기반 검색(BM25, TF-IDF)
- 6단계 프롬프트: RAG 를 수행하기 위한 프롬프트를 생성합니다. 프롬프트의 context 에는 문서에서 검색된 내용이 입력됩니다. 프롬프트 엔지니어링을 통하여 답변의 형식을 지정할 수 있습니다.
- 7단계 LLM: 모델을 정의합니다.(GPT-3.5, GPT-4, Claude, etc..)
- 8단계 Chain(생성): 프롬프트 - LLM - 출력 에 이르는 체인을 생성합니다. (텍스트, JSON, 마크다운)

# Langchain Q&A 아키텍처

LangChain Q&A 애플리케이션의 아키텍처는 크게 질문 처리와 답변 생성 두 가지 주요 단계로 구성됩니다. 이 과정에서 여러 LangChain 구성 요소가 사용됩니다.

① 질문 처리 Permalink
- 사용자의 질문을 받아 이를 처리
- 관련 데이터를 찾는 작업
- 필요한 구성 요소:
    - 데이터 소스 연결: 질문에 대한 답변을 찾기 위해 다양한 텍스트 데이터 소스에 연결해야 합니다. LangChain은 다양한 데이터 소스와의 연결을 간편하게 설정할 수 있도록 돕습니다.
    - 데이터 인덱싱 및 검색: 데이터 소스에서 관련 정보를 효율적으로 찾기 위해, 데이터는 인덱싱되어야 합니다. LangChain은 인덱싱 과정을 자동화하고, 사용자의 질문과 관련된 데이터를 검색하는 데 필요한 도구를 제공합니다.

② 답변 생성 Permalink
- 필요한 구성 요소:
    - 답변 생성 모델: LangChain은 고급 자연어 처리(NLP) 모델을 사용하여 검색된 데이터로부터 답변을 생성할 수 있는 기능을 제공합니다. 이러한 모델은 사용자의 질문과 검색된 데이터를 입력으로 받아, 적절한 답변을 생성합니다.

LangSmith: LangSmith는 애플케이션의 성능을 추적하고 분석하는 데 도움을 주는 도구입니다. Q&A 애플리케이션의 복잡성이 증가함에 따라, LangSmith는 다음과 같은 방법으로 유용하게 사용될 수 있습니다:
- 성능 모니터링: LangSmith를 사용하여 애플리케이션의 질문 처리 및 답변 생성 성능을 모니터링할 수 있습니다. 이를 통해 애플리케이션의 성능을 지속적으로 개선할 수 있습니다.
- 오류 분석: LangSmith는 애플리케이션이 잘못된 답변을 생성했을 때, 이의 원인을 분석하는 데 도움을 줍니다. 이를 통해 개발자는 애플리케이션의 오류를 정확하게 파악하고, 이를 수정하는 데 필요한 정보를 얻을 수 있습니다.

# 환경설정


API KEY 를 설정합니다.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

LangChain으로 구축한 애플리케이션은 여러 단계에 걸쳐 LLM 호출을 여러 번 사용하게 됩니다. 이러한 애플리케이션이 점점 더 복잡해짐에 따라, 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사할 수 있는 능력이 매우 중요해집니다. 이를 위한 최선의 방법은 [LangSmith](https://smith.langchain.com)를 사용하는 것입니다.

LangSmith가 필수는 아니지만, 유용합니다. LangSmith를 사용하고 싶다면, 위의 링크에서 가입한 후, 로깅 추적을 시작하기 위해 환경 변수를 설정해야 합니다.


In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


# 네이버 뉴스 기반 QA(Question-Answering) 챗봇

이번 튜토리얼에는 네이버 뉴스기사의 내용에 대해 질문할 수 있는 **뉴스기사 QA 앱** 을 구축할 것입니다.

이 가이드에서는 OpenAI 챗 모델과 임베딩, 그리고 Chroma 벡터 스토어를 사용할 것입니다.

먼저 다음의 과정을 통해 간단한 인덱싱 파이프라인과 RAG 체인을 약 20줄의 코드로 구현할 수 있습니다.

라이브러리

- `bs4`는 웹 페이지를 파싱하기 위한 라이브러리입니다.
- `langchain`은 AI와 관련된 다양한 기능을 제공하는 라이브러리로, 여기서는 특히 텍스트 분할(`RecursiveCharacterTextSplitter`), 문서 로딩(`WebBaseLoader`), 벡터 저장(`Chroma`, `FAISS`), 출력 파싱(`StrOutputParser`), 실행 가능한 패스스루(`RunnablePassthrough`) 등을 다룹니다.
- `langchain_openai` 모듈을 통해 OpenAI의 챗봇(`ChatOpenAI`)과 임베딩(`OpenAIEmbeddings`) 기능을 사용할 수 있습니다.


In [3]:
import bs4 #웹페이지 파싱
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter #텍스트 분할
from langchain_community.document_loaders import WebBaseLoader #문서 로딩
from langchain_community.vectorstores import FAISS #벡터 저장
from langchain_core.output_parsers import StrOutputParser #출력 파싱
from langchain_core.runnables import RunnablePassthrough #실행 가능한 패스스루
from langchain_openai import ChatOpenAI, OpenAIEmbeddings #open ai 챗봇 #임베딩 기능

USER_AGENT environment variable not set, consider setting it to identify your requests.


(1) 웹 페이지의 내용을 로드하고, (2) 텍스트를 청크로 나누어 인덱싱하는 과정을 거친 후, (3) 관련된 텍스트 스니펫을 검색하여 (4) 새로운 내용을 생성하는 과정을 구현합니다.

`WebBaseLoader`는 지정된 웹 페이지에서 필요한 부분만을 파싱하기 위해 `bs4.SoupStrainer`를 사용합니다.

[참고]

- `bs4.SoupStrainer` 는 편리하게 웹에서 원하는 요소를 가져올 수 있도록 해줍니다.

(예시)

```python
bs4.SoupStrainer(
    "div",
    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
)
```


### 1. 뉴스 기사 내용 로드

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 

**뉴스 기사 로드**
- WebBaseLoader는 웹 페이지에서 텍스트 데이터를 가져오는 LangChain의 기본 로더.
- web_paths에 네이버 뉴스 기사 URL을 전달하여 해당 페이지를 크롤링함.
- ("url",)처럼 튜플 형태로 전달해야 함 (, 필수

**bw_kwargs로 필요한 부분만 가져옴**
- bs_kwargs: BeautifulSoup의 SoupStrainer를 사용하여 특정 HTML 태그만 필터링.
- parse_only=bs4.SoupStrainer(...):
- "div" 태그 중 class가 "newsct_article _article_body" 또는 "media_end_head_title"인 요소만 추출. 즉, 뉴스 기사 본문과 제목만 가져옴.

**div**
- 이 구조에 맞춰 div 태그에서 기사 제목과 본문만 가져오도록 설정

**문서 로드**
- loader.load()를 실행하면 네이버 뉴스 페이지에서 HTML을 가져와서, 설정한 부분만 추출한 후 문서 리스트로 변환함.
docs는 LangChain의 Document 객체 리스트가 됨.


### 2. chunk 나누기

`RecursiveCharacterTextSplitter`는 문서를 지정된 크기의 청크로 나눕니다.


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

### 3. 인덱싱

`FAISS` 혹은 `Chroma`와 같은 vectorstore는 이러한 청크를 바탕으로 문서의 벡터 표현을 생성합니다.


In [6]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

`vectorstore.as_retriever()`를 통해 생성된 검색기는 `hub.pull`로 가져온 프롬프트와 `ChatOpenAI` 모델을 사용하여 새로운 내용을 생성합니다.

마지막으로, `StrOutputParser`는 생성된 결과를 문자열로 파싱합니다.


In [7]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

hub 에서 `teddynote/rag-prompt-korean` 프롬프트를 다운로드 받아 입력할 수 있습니다. 이런 경우 별도의 프롬프트 작성과정이 생략됩니다.

In [15]:
prompt = hub.pull("teddynote/rag-prompt-korean")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'rag-prompt-korean', 'lc_hub_commit_hash': '4214de29c72c82d13121c710b0ee7021d31ad088f0eb1d6ddebcaaeab46b1f8b'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.\n검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.\n한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step.")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='#Question: \n{question} \n\n#Context: \n{context} \n\n#Answer:'))])

In [ ]:
## 블로그와 다른 지점
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

스트리밍 출력을 위하여 `stream_response` 를 사용합니다.

In [17]:
from langchain_teddynote.messages import stream_response

> [LangSmith Trace 보기](https://smith.langchain.com/public/c6047a61-8f44-48e5-89eb-b1e8a6321cea/r)


In [18]:
answer = rag_chain.stream("부영그룹의 출산 장려 정책에 대해 설명해주세요.")
stream_response(answer)

부영그룹의 출산 장려 정책은 다음과 같습니다. 2021년 이후 태어난 직원 자녀에게 1억원씩 지원하며, 연년생과 쌍둥이 자녀가 있는 경우 총 2억원을 받을 수 있습니다. 또한, 셋째 아이를 낳는 경우 국민주택을 제공하겠다는 계획도 있습니다. 이 정책은 출산한 직원에게 금전적인 지원을 통해 출산을 장려하고자 하는 목적을 가지고 있습니다.

> [LangSmith Trace 보기](https://smith.langchain.com/public/ed21d80e-b4da-4a08-823b-ed980db9c347/r)


In [19]:
answer = rag_chain.stream("부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?")

> [LangSmith Trace 보기](https://smith.langchain.com/public/df80c528-61d6-4c83-986a-3373a4039dae/r)


In [20]:
answer = rag_chain.stream("정부의 저출생 대책을 bullet points 형식으로 작성해 주세요.")
stream_response(answer)

- 부모 급여: 0세 아이에게 매달 100만원 지급
- 첫만남이용권 및 아동수당 제공
- 아이가 돌까지 1년 동안 총 1520만원 지원
- 인천시: 새로 태어난 아기에게 18살 될 때까지 1억원 지급
- 광주시: 17살 될 때까지 7400만원 지급
- 기업의 출산장려금: 출산한 직원에게 1억원 지급
- 셋째 아이 출산 시 국민주택 제공
- 육아휴직 기간 연장 및 남성 직원의 육아휴직 의무화
- 사내 어린이집 운영 시간 연장 (밤 10시까지)

> [LangSmith Trace 보기](https://smith.langchain.com/public/1a613ee7-6eaa-482f-a45f-8c22b4e60fbf/r)


In [21]:
answer = rag_chain.stream("부영그룹의 임직원 숫자는 몇명인가요?")
stream_response(answer)

주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.

### 블로그 작성 버전

In [24]:
## 블로그와 다른 지점
llm2 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 체인을 생성합니다.
rag_chain2 = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm2
    | StrOutputParser()
)

`format_docs` 함수
- docs(검색된 문서 리스트)에서 문서 본문(page_content)만 추출하여 하나의 문자열로 합치는 역할
- 이렇게 정리된 문서를 LLM에 전달하여 질문과 함께 처리할 수 있게 됨
- invoke()는 한 번에 데이터를 처리해야 하기 때문에, 문서를 미리 정리해서 전달하는 것이 중요

In [25]:
rag_chain2.invoke(
    "부영그룹의 출산 장려 정책에 대해 설명해주세요."
)

'부영그룹은 2021년 이후 태어난 직원 자녀에게 1억원을 지원하고, 연년생과 쌍둥이 자녀가 있는 경우에는 총 2억원을 받게 됩니다. 또한, 셋째까지 낳는 경우에는 국민주택을 제공할 계획이 있습니다.'

In [26]:
rag_chain2.invoke(
    "부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?"
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

'부영그룹은 출산 직원에게 1억원의 지원을 제공합니다.'

In [27]:
print(
    rag_chain2.invoke("정부의 저출생 대책을 bullet points 형식으로 작성해 주세요.")
)

- 정부가 매달 주는 부모 급여를 0세 아이는 100만원으로 올림
- 지자체에서는 새로 태어난 아기에게 1억원을 주는 등 현금 지원을 실시
- 기업들도 출산한 직원에게 1억원을 주는 등 파격적인 정책을 시행함
- 회사가 출산장려금을 제공할 경우 정부가 면세 혜택을 고려해달라는 제안이 나옴
- 출산장려책이 점점 확산되고 있으며, 육아휴직을 확대하거나 남성 직원의 육아휴직을 의무화하는 등의 조치가 이루어지고 있음


In [29]:
# 컬렉션을 삭제합니다.*
vectorstore.delete_collection()

AttributeError: 'FAISS' object has no attribute 'delete_collection'

vectorstore 객체의 delete_collection 메서드를 호출하여 컬렉션을 삭제합니다.

이는 데이터 정리 과정에서 사용되며, 특정 데이터셋이나 정보를 저장하는 컬렉션을 제거함으로써 시스템의 저장 공간을 확보하고, 불필요한 데이터로 인한 혼란을 방지합니다.

# Basic-Webloader-Practice

In [30]:
loader = WebBaseLoader(
    web_paths=("https://www.nytimes.com/2025/02/25/magazine/morning-ritual-poetry-memorization.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(metadata={'source': 'https://www.nytimes.com/2025/02/25/magazine/morning-ritual-poetry-memorization.html'}, page_content='')]

In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

0

In [35]:
# ✅ 크롤링한 문서에서 내용이 없는 문서 제거
docs = [doc for doc in docs if doc.page_content.strip()]

# ✅ 문서를 청크로 나누기 (비어 있는 문서는 제거됨)
splits = text_splitter.split_documents(docs)

print(f"유효한 문서 청크 개수: {len(splits)}")  # ✅ 최종 청크 개수 확인

if not splits:
    raise ValueError("🚨 청크된 문서가 없습니다! 다시 크롤링을 실행하세요.")

# ✅ 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


유효한 문서 청크 개수: 0


ValueError: 🚨 청크된 문서가 없습니다! 다시 크롤링을 실행하세요.

In [32]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

IndexError: list index out of range

`vectorstore.as_retriever()`를 통해 생성된 검색기는 `hub.pull`로 가져온 프롬프트와 `ChatOpenAI` 모델을 사용하여 새로운 내용을 생성합니다.

마지막으로, `StrOutputParser`는 생성된 결과를 문자열로 파싱합니다.


In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

hub 에서 `teddynote/rag-prompt-korean` 프롬프트를 다운로드 받아 입력할 수 있습니다. 이런 경우 별도의 프롬프트 작성과정이 생략됩니다.

In [ ]:
prompt = hub.pull("teddynote/rag-prompt-korean")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'rag-prompt-korean', 'lc_hub_commit_hash': '4214de29c72c82d13121c710b0ee7021d31ad088f0eb1d6ddebcaaeab46b1f8b'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.\n검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.\n한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step.")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='#Question: \n{question} \n\n#Context: \n{context} \n\n#Answer:'))])

In [ ]:
## 블로그와 다른 지점
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

스트리밍 출력을 위하여 `stream_response` 를 사용합니다.

In [ ]:
from langchain_teddynote.messages import stream_response

> [LangSmith Trace 보기](https://smith.langchain.com/public/c6047a61-8f44-48e5-89eb-b1e8a6321cea/r)


In [ ]:
answer = rag_chain.stream("부영그룹의 출산 장려 정책에 대해 설명해주세요.")
stream_response(answer)

부영그룹의 출산 장려 정책은 다음과 같습니다. 2021년 이후 태어난 직원 자녀에게 1억원씩 지원하며, 연년생과 쌍둥이 자녀가 있는 경우 총 2억원을 받을 수 있습니다. 또한, 셋째 아이를 낳는 경우 국민주택을 제공하겠다는 계획도 있습니다. 이 정책은 출산한 직원에게 금전적인 지원을 통해 출산을 장려하고자 하는 목적을 가지고 있습니다.